<a href="https://colab.research.google.com/github/YoungHyunKoo/GEE_Fall2025/blob/main/3_Image_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **[GEO 6083] Remote Sensing Imge Processing - Fall 2025**
# **MODULE 3. Image analysis**

### OBJECTIVES
1. Undestand and calculate NDVI and other indices.
2. Create time series of Landsat images.
3. Calculate regional statistics.
4. Extract time series for a region of interest using EarthEngine image collections.

Credited by Younghyun Koo (kooala317@gmail.com)


## **1. Index calculation**

### **How do math operations (or functions) work in GEE?**
Earth Engine supports many basic mathmatical operators (e.g. `add`, `subtract`, `multiply`, `divide`, etc.). Earth Engine performs these math operators by pixel. When an operator is applied to a single image, it's applied to pixels of each band. In the case of operations on two different images, the operation is only applied at the location where pixels in both images are unmasked. When an operator is applied to two images, the images are expected to have the same number of bands and spatial resolutions so they can be matched pairwise. However, if one of the images has only a single band, it is matched with all of the bands in the other image, essentially replicating that band enough time to match the other image.

### **Import image collections**

In [ ]:
# Import ee library
import ee

try:
  # Initialize with your own project.
  ee.Initialize(project = "utsa-spring2024")
except:
  # Authenticate
  ee.Authenticate()
  # Initialize with your own project.
  ee.Initialize(project = "utsa-spring2024")

In [ ]:
# Import geemap library
import geemap

# Import os library
import os

In this example, we will use some optical satellite images to observe a destructive and deadly flooding event in Texas in the summer of 2025. This flood killed at least 135 people (117 people in Kerr County), including 27 young kids at Camp Mystic. For this flood monitoring, we will use Sentinel-2 optical imagery, which has a finer spatial resolution (10 m) than Landsat (30 m): [Sentinel-2 MSI: MultiSpectral Instrument, Level-2A](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED).

Let's import a image collection for a specific region and time we are interested in. Also, we will filter out the images with cloud covers less than 10 %. We already did how to filter out an image collection using `filterDate`, `filterBounds`, and `filterMetadata` functions.

In [ ]:
# Define region of interest (around the Kerr County)
roi = ee.Geometry.Point(-99.3702, 30.0091)

# Import Sentinel-2 optical image (before flood)
S2_1 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterDate('2025-06-10', '2025-06-30') \
    .filterBounds(roi) \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)

# Import Sentinel-2 optical image (after flood)
S2_2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterDate('2025-07-01', '2025-07-30') \
    .filterBounds(roi) \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)

print(S2_1.size().getInfo(), S2_2.size().getInfo())

Let's visualize the true color images on the map.

In [ ]:
# Load a geemap
Map = geemap.Map(basemap='HYBRID') # Change basemap setting to satellite imagery

# image visualization factors
vis_param = {'min': 0,
             'max': 3000,
             'bands': ['B4', 'B3', 'B2'],
             'gamma': 1.0}

image1 = S2_1.first()
image2 = S2_2.first()

Map.addLayer(image1, vis_param, "Before flood")
Map.addLayer(image2, vis_param, "After flood")
Map.addLayer(roi, {'color': 'red'})
Map.centerObject(roi, 13)

Map

### **Use functions to calculate NDVI**

Okay, from true-color images, we found that this flood event changed the land cover significantly, particularly around the Guadalupe River. Around the Guadalupe River, many green vegetation areas were swept away by the flooding. Now, we will further investigate and analyze this flood-driven land cover change by calculating NDVI (Normalized Difference Vegetation Index). NDVI quantified vegetation by measuring the difference between near-infrared (which vegetation strongly reflects) and red light (which vegetation absorbs). For more details about NDVI, please visit this website: [link](https://earthobservatory.nasa.gov/features/MeasuringVegetation/measuring_vegetation_2.php). NDVI ranges from -1 to 1.

<img src="https://ece.montana.edu/seniordesign/archive/SP15/OpticalWeedMapping/uploads/4/9/2/7/49273335/1429843234.png" width="400">

For Sentinel-2 images, band 5 corresponds to NIR band and band 4 is red band ([Sentinel-2 MSI: MultiSpectral Instrument, Level-2A](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED)). You can use **mathematical expressions** to calculate any index. By using `image.expression()`, you can parse a text representation of a math operation.

In [ ]:
# Calculate NDVI of the first image using mathmetical expression
NDVI1 = image1.expression(
    '(NIR - RED) / (NIR + RED)', {
      'NIR': image1.select('B5'),
      'RED': image1.select('B4')
})

# Calculate NDVI of the second image using mathmetical expression
NDVI2 = image2.expression(
    '(NIR - RED) / (NIR + RED)', {
      'NIR': image2.select('B5'),
      'RED': image2.select('B4')
})

As you can see in the previous cell, expression requires a few arguments

(1) Textual representation of the math operation: `(NIR - RED) / (NIR + RED)`

(2) Dictionary where the keys are variable names used in the expression and the values are the image bands to which the variables should be mapped: `{'NIR': image.select('B5'), 'RED': image.select('B4')}`

Let's visualize these NDVI results on the map.

In [ ]:
# Load a geemap
Map = geemap.Map(basemap='HYBRID')

# NDVI visualization factors (min, max, and palette)
ndvi_param = {'min': -1, 'max': 1,
             'palette': ['red', 'white', 'green']}

# Visualize NDVI maps
Map.addLayer(NDVI1, ndvi_param, "NDVI 1")
Map.addLayer(NDVI2, ndvi_param, "NDVI 2")

# Visualize NDVI difference
Map.addLayer(NDVI2.subtract(NDVI1), ndvi_param, "Diff")

# Location of Camp Mystic
Map.addLayer(roi, {'color': 'red'})

Map.centerObject(roi, 13)
Map

### **Another example: monitoring wildfire using Landsat**

NDVI calculation can also be useful for monitoring wildfire events. We will take a look at a large wildfire event near Mendocino County, Northern California, in July 2018 (*Mendocino Complex Fire*). You can get more information about this wildfire in this link [Medocino Complex Fire](https://en.wikipedia.org/wiki/Mendocino_Complex_Fire). After the fire, all vegetation covers are burned so there should be decrease in vegetation index. We will monitor this wildfire event through the variations of NDVI value near the Mendocino County before and after the wildfire.

Let's see what really happened in this region using the Landsat images.

In [ ]:
# Point of interest: around Mendocino County
poi = ee.Geometry.Rectangle([[-122.85, 39.15], [-122.75, 39.25]])

# Landsat image before the fire
ls1 = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
    .filterDate('2018-07-01', '2018-07-31') \
    .filterBounds(poi) \
    .filterMetadata('CLOUD_COVER', 'less_than', 10) \
    .sort("CLOUD_COVER").first()

# Landsat image after the fire
ls2 = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
    .filterDate('2019-07-01', '2019-07-31') \
    .filterBounds(poi) \
    .filterMetadata('CLOUD_COVER', 'less_than', 10) \
    .sort("CLOUD_COVER").first()

In [ ]:
# Load a geemap
Map = geemap.Map()

# image visualization factors
vis_param = {'min': 0,
             'max': 0.3,
             'bands': ['B4', 'B3', 'B2'],
             'gamma': 1.0}

Map.addLayer(ls1, vis_param, "Before fire")
Map.addLayer(ls2, vis_param, "After fire")
Map.addLayer(poi, {}, "ROI")

Map.centerObject(poi, 10)

Map

In [ ]:
# Calculate NDVI of the first image using mathmetical expression
NDVI1 = ls1.expression(
    '(NIR - RED) / (NIR + RED)', {
      'NIR': ls1.select('B5'),
      'RED': ls1.select('B4')
})

# Calculate NDVI of the second image using mathmetical expression
NDVI2 = ls2.expression(
    '(NIR - RED) / (NIR + RED)', {
      'NIR': ls2.select('B5'),
      'RED': ls2.select('B4')
})

In [ ]:
# NDVI visualization factors (min, max, and palette)
ndvi_param = {'min': -1, 'max': 1,
             'palette': ['red', 'white', 'green']}

# Visualize NDVI maps
Map.addLayer(NDVI1, ndvi_param, "NDVI 1")
Map.addLayer(NDVI2, ndvi_param, "NDVI 2")

# Visualize NDVI difference
Map.addLayer(NDVI2.subtract(NDVI1), ndvi_param, "Diff")

Map.centerObject(poi, 13)
Map

Now, let's make a time-series animation around this wildfire area using Landsat.

In [ ]:
# Install an additional phthon package for this tutorial
!pip install ffmpeg-python

In [ ]:
# Import the package we just installed
import ffmpeg

We will create a timelapse animation to see the urban growth in Las Vegas area. We will use the fuction named `add_landsat_ts_gif` of `geemap` library. First, we will define the area of interest we want to visualize.
[How to define a polygon](https://developers.google.com/earth-engine/apidocs/ee-geometry-polygon)

In [ ]:
# Area of interest: around Mendocino County
roi = ee.Geometry.Rectangle([[-123.6, 38.8], [-122.0, 40.2]])

In [ ]:
# Import image collection - Landsat 8 surface reflectance
Map = geemap.Map()
Map

label = 'Wildfires around the Mendocino County'
Map.add_landsat_ts_gif(
    label=label,
    start_year = 2000,
    end_year = 2025,
    start_date='05-01',
    end_date='09-30',
    bands=['SWIR1', 'NIR', 'Blue'],
    font_color='white',
    frames_per_second=1,
    progress_bar_color='blue',
    roi = roi
)

Map.centerObject(roi, 9)
Map

Similarly, let's create a timelapse animation for San Antonio area.

In [ ]:
# Area of interest: San Antonio area
roi = ee.Geometry.Rectangle([[-98.8, 29.2], [-98.2, 29.6]])

In [ ]:
# Import image collection - Landsat 8 surface reflectance
Map = geemap.Map()
Map

label = 'Urban Growth in San Antonio'
Map.add_landsat_ts_gif(
    label=label,
    start_year = 2000,
    end_year = 2025,
    bands=['Red', 'Green', 'Blue'], # NIR, Red, Green
    font_color='white',
    frames_per_second=2,
    progress_bar_color='blue',
    roi = roi
)

Map.centerObject(roi, 10)
Map

Now, let's take a look at another huge wildfire, which happened near in Los Angeles this January (Palisades Fire).

In [ ]:
poi = ee.Geometry.Point(-118.53, 34.10)

ls1 = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
    .filterDate('2024-11-01', '2024-12-31') \
    .filterBounds(poi) \
    .filterMetadata('CLOUD_COVER', 'less_than', 10) \
    .sort("CLOUD_COVER").first()

ls2 = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
    .filterDate('2025-01-01', '2025-03-31') \
    .filterBounds(poi) \
    .filterMetadata('CLOUD_COVER', 'less_than', 10) \
    .sort("CLOUD_COVER").first()

In [ ]:
ls1

In [ ]:
# Load a geemap
Map = geemap.Map()

# image visualization factors
vis_param = {'min': 0,
             'max': 0.2,
             'bands': ['B4', 'B3', 'B2'],
             'gamma': 1.0}

Map.addLayer(ls1, vis_param, "Before fire")
Map.addLayer(ls2, vis_param, "After fire")
Map.addLayer(poi, {}, "ROI")

Map.centerObject(poi, 12)

Map

In [ ]:
# Calculate NDVI of the first image using mathmetical expression
NDVI1 = ls1.expression(
    '(NIR - RED) / (NIR + RED)', {
      'NIR': ls1.select('B5'),
      'RED': ls1.select('B4')
})

# Calculate NDVI of the second image using mathmetical expression
NDVI2 = ls2.expression(
    '(NIR - RED) / (NIR + RED)', {
      'NIR': ls2.select('B5'),
      'RED': ls2.select('B4')
})

In [ ]:
# NDVI visualization factors (min, max, and palette)
ndvi_param = {'min': -1, 'max': 1,
             'palette': ['red', 'white', 'green']}

# Visualize NDVI maps
Map.addLayer(NDVI1, ndvi_param, "NDVI 1")
Map.addLayer(NDVI2, ndvi_param, "NDVI 2")

# Visualize NDVI difference
Map.addLayer(NDVI2.subtract(NDVI1), ndvi_param, "Diff")

# Location of Camp Mystic
Map.addLayer(poi, {'color': 'red'})

Map.centerObject(poi, 13)
Map

***DO IT YOURSELF!!***
- You can see some other indices as well in this website: [Remote Sensing Indices](https://medium.com/regen-network/remote-sensing-indices-389153e3d947). Please try and visualize any index you want to calculate using expressions.

In [ ]:
# Try your own index here


## **2. Get image statistcis by region**

You are also able to calculate the statistics of image for a certain region of interst. Here we will calculate the NDVI values for a certain state/county and compare each state and county.

- US Census 2018 states data [LINK](https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_States)
- US Census 2018 counties data: [LINK](https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_Counties#table-schema)

In [ ]:
# See state feature collection
states = ee.FeatureCollection('TIGER/2018/States').filter(ee.Filter.lt("REGION", "9"))
# filter 50 states + DC

Map = geemap.Map()

Map.addLayer(states)
Map.centerObject(states, 4)
Map

In [ ]:
# Feature collection to pandas table
states_table = geemap.ee_to_df(states)

states_table

In [ ]:
# See Texas state feature
Texas = ee.FeatureCollection('TIGER/2018/States').filter(ee.Filter.eq("STATEFP", "48"))

Map = geemap.Map()

Map.addLayer(Texas)
Map.centerObject(Texas, 6)
Map

In [ ]:
# See Texas counties feature collection
counties_TX = ee.FeatureCollection('TIGER/2018/Counties').filter(ee.Filter.eq("STATEFP", "48"))

Map = geemap.Map()

Map.addLayer(counties_TX)
Map.centerObject(counties_TX, 6)
Map

In [ ]:
# See Bexar county feature collection
Bexar = ee.FeatureCollection('TIGER/2018/Counties').filter(ee.Filter.eq("NAME", "Bexar"))

Map = geemap.Map()

Map.addLayer(Bexar)
Map.centerObject(Bexar, 8)
Map

### **Reduceregion**

Now we will use `reduceRegion` function to extract the mean NDVI of Texas. [reduceRegion](https://developers.google.com/earth-engine/apidocs/ee-image-reduceregion)

For NDVI, we will use the MODIS NDVI image collection. Although MODIS has a coarse resoltuion (500 m - 1 km) compared to Landsat, it provides global scale observations for a large area with a daily time scale. The daily NDVI data from MODIS is available in GEE (16-days average). Please see the details of this dataset via this link: [MODIS Terra NDVI](https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD13A1)

In [ ]:
# Extract the mean NDVI value in June (note: multiply scale factor to convert digital number to NDVI)
ndvi = ee.ImageCollection('MODIS/061/MOD13A1').filterDate("2018-06-01", "2018-06-30").mean().select("NDVI").multiply(0.0001)

ndvi_TX = ndvi.reduceRegion(
    geometry = Texas.geometry(), # geometry to Texas
    reducer = ee.Reducer.mean(),
    scale = 1000,
    maxPixels = 1e9
)

# Output of reduceRegion: ee.dictionary
print(ndvi_TX.getInfo()['NDVI'])

We can do the same thing for the Bexar County.

In [ ]:
# Extract the mean NDVI value in June (note: multiply scale factor to convert digital number to NDVI)
ndvi = ee.ImageCollection('MODIS/061/MOD13A1').filterDate("2018-06-01", "2018-06-30").mean().select("NDVI").multiply(0.0001)

# reduceRegion function for Bexar County
ndvi_Bexar = ndvi.reduceRegion(
    geometry = Bexar.geometry(), # geometry to Bexar County
    reducer = ee.Reducer.mean(),
    scale = 1000,
    maxPixels = 1e9 # The maximum number of pixels to reduce (should be large enough to cover the entire Bexar County).
)

# Output of reduceRegion: ee.dictionary
print(ndvi_Bexar.getInfo()['NDVI'])

### **Reduceregions**

If you want to compare multiple features in the feature collection, we can use `reduceRegions` function. [reduceRegions](https://developers.google.com/earth-engine/apidocs/ee-image-reduceregions) This function will allow us to extract the reduce results for multiple states or counties.

In [ ]:
# reduceRegions function: multiple counties
ndvi_counties = ndvi.reduceRegions(
    collection = counties_TX,
    reducer = ee.Reducer.mean(),
    scale = 1000
)

# Output of reduceRegions: feature collection
df_ndvi_counties = geemap.ee_to_df(ndvi_counties)
df_ndvi_counties.head(10)

In [ ]:
# Check the information for Bexar County
df_ndvi_counties[df_ndvi_counties["NAME"] == "Bexar"]

Now let's do the same thing for the MODIS land surface temperature data. [MODIS Terra Land Surface Temperature and Emissivity 8-Day Global 1km ](https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD11A2)

In [ ]:
# Land surface temperature on June 10, 2018
temp = ee.ImageCollection("MODIS/061/MOD11A2").filterDate("2018-06-01", "2018-06-30").mean().select("LST_Day_1km").multiply(0.02)
# Multiply 0.02 to convert DN to K

In [ ]:
# Multiple counties
temp_counties = temp.reduceRegions(
    collection = counties_TX,
    reducer = ee.Reducer.mean(),
    scale = 1000
)

# Output of reduceRegions: feature collection
df_temp_counties = geemap.ee_to_df(temp_counties)
df_temp_counties.head(10)

### **Plot the relationship with NDVI v. temperature**

From this result, let's draw a plot to compare the relationships between NDVI and surface temperature.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(df_ndvi_counties['mean'], df_temp_counties['mean'])
plt.xlabel("NDVI")
plt.ylabel("Temperature (K)")
plt.title("NDVI v. Temperature for TX counties")

On the above scater plot, each point represent NDVI and LST for each county in Texas.

Now let's draw the scatter plot between latitude and temperature to see if these two variables have any significant correlations.

In [ ]:
from scipy import stats
import numpy as np

res = stats.linregress(df_ndvi_counties['mean'], df_temp_counties['mean'])

a = res.slope
b = res.intercept

print(a, b)

In [ ]:
# Draw a scatter plot between NDVI and surface temperature
plt.scatter(df_ndvi_counties['mean'], df_temp_counties['mean'])
plt.xlabel("NDVI")
plt.ylabel("Temperature (K)")

# Linear regression line
x = np.arange(df_ndvi_counties['mean'].min(), df_ndvi_counties['mean'].max(), 0.01)
plt.plot(x, a*x + b, color = 'k', ls = "--", lw = 2)

plt.title("NDVI v. Temperature for US states")

## **3. Get time-series of region**

Google Earth Engine has a significant advantage in processing lots of satellite images without downloading every single file. In this example, we will extract the time series of surface temperature in Bexar County using MODIS data.

In [ ]:
# MODIS Land surface temperature image collection
temp = ee.ImageCollection("MODIS/061/MOD11A2")

In [ ]:
import pandas as pd

def temp_mean(img):
    mean = img.reduceRegion(reducer=ee.Reducer.mean(), geometry=Bexar.geometry(), scale=1000).get('LST_Day_1km')
    return img.set('date', img.date().format()).set('temp',mean)

In [ ]:
bexar_reduced_imgs = temp.map(temp_mean)
bexar_temp_list = bexar_reduced_imgs.reduceColumns(ee.Reducer.toList(2), ['date','temp']).values().get(0)

In [ ]:
df_temp = pd.DataFrame(bexar_temp_list.getInfo(), columns=['date', 'temp'])
df_temp['date'] = pd.to_datetime(df_temp['date'])
df_temp['temp'] = df_temp['temp'] * 0.02
df_temp

### **Draw time series plot**

Now, let's draw the time series plot of surface temperature using the `matplotlib` package.

In [ ]:
plt.subplots(1, 1, figsize = (12,5), dpi = 100)
plt.plot(df_temp['date'], df_temp['temp'])
plt.xlabel("Date")
plt.ylabel("Temperature (K)")

Can you find any abnormal temperature? You can find some information about this abnormal temperature event at the link below:

[**What happened?**](https://www.ncei.noaa.gov/news/great-texas-freeze-february-2021)

## **References**
- https://geemap.org/tutorials/#geemap-tutorials
- https://developers.google.com/earth-engine/apidocs/ee-image-reduceregions